In [41]:
# Why Did They Actually Import This ????? of no use.....
# from __future__ import print_function

In [104]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]
header = ['color', 'diameter', 'label']

In [105]:
def unique_values(td, col):
    
    return set(td[i][col] for i in range(len(td)))

def class_count(td):
    
    labels = {}
    for row in td:
        if row[-1] not in labels:
            labels[row[-1]] = 0
        labels[row[-1]] += 1
    return labels    
            
def is_numeric(x):
    
    return isinstance(x, int) or isinstance(x, float)

In [106]:
class Question: 
    
    def __init__(self, column, value): #value is basically the threshold # And the column tells which feature
        self.column = column
        self.value = value
        
    def match(self, example):
        num = is_numeric(example[self.column])
        if num:
            return example[self.column] >= self.value
        else:
            return example[self.column]== self.value
    
    def __repr__(self):
        num = is_numeric(self.value)
        condition = "=="
        if num:
            condition = ">="
        return f"is {header[self.column]} {condition} {self.value} ?"
    

In [107]:
def partition(td, question):
    
    true_rows, false_rows = [], []
    if is_numeric(question.value):
        for row in td:
            if row[question.column] >= question.value:
                true_rows.append(row)
            else:
                false_rows.append(row)
    else:
        for row in td:
            if row[question.column] == question.value:
                true_rows.append(row)
            else:
                false_rows.append(row)
    return true_rows, false_rows

In [108]:
def gini(td):
    
    impurity = 1
    labels = class_count(td)
    for label in labels:
        impurity -= (labels[label]/len(td))**2
    return impurity    

In [109]:
def info_gain(left, right, current_uncertainity):
    
    p = len(left)/(len(right)+len(left))
    return current_uncertainity - p*(gini(left)) - (1-p)*(gini(right))

In [116]:
def find_best_split(td):
    
    n_features = len(td[0])-1
    best_gain = 0 # Helpful To Identify The Leaf Node In Build_Tree
    best_question = None
    current_uncertainity = gini(td)
    for col in range(n_features):
        uniques = unique_values(training_data, col)
        for f in uniques:
            question = Question(col, f)
            true_rows, false_rows = partition(td, question)
            if len(true_rows) == 0 or len(false_rows) == 0:  # If Leaf Node Then Dont Do Any Splitting
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            if gain >= best_gain: 
                best_gain = gain
                best_question = question
    return best_gain, best_question

class Leaf:
    
    def __init__(self, td):
        self.predictions = class_count(td)
        
class Decision_Node:
    
    def __init__(self, question, true_branch, false_branch):
        
        self.question = question
        self.true_branch = true_branch         
        self.false_branch = false_branch

def build_tree(td):
    
    gain, question = find_best_split(td)
    if gain == 0:
        return Leaf(td)
    true_branch, false_branch = partition(td, question)
    true_branch = build_tree(true_branch)
    false_branch = build_tree(false_branch)
    return Decision_Node(question, true_branch, false_branch)

def print_tree(node, space = ""):
    
    if isinstance(node, Leaf):
        print(space + 'predict' ,node.predictions)
        return 
    print(space + str(node.question))
    print(space + '--> True:')
    print_tree(node.true_branch, space = "  ")
    print(space + '--> False:')
    print_tree(node.false_branch, space = "  ") 

def classify(example, node):
    
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(example):
        return classify(example, node.true_branch)
    else:
        return classify(example, node.false_branch)    

def print_leaf(leaf):
    
    probs = {}
    total = sum(int(leaf[label]) for label in leaf)
    for label in leaf:
        probs[label] = str(int(leaf[label]/total  * 100))  + "%"
    return probs

# Final-----Output------Is------Here

In [122]:
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
for row in testing_data:
    
    print(f"Actual {row[-1]} and predicted {print_leaf(classify(row, build_tree(training_data)))}")

Actual Apple and predicted {'Apple': '100%'}
Actual Apple and predicted {'Apple': '50%', 'Lemon': '50%'}
Actual Grape and predicted {'Grape': '100%'}
Actual Grape and predicted {'Grape': '100%'}
Actual Lemon and predicted {'Apple': '50%', 'Lemon': '50%'}
